In [1]:
import numpy as np
import pandas as pd
import scipy.stats

pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt, rcParams
# import cv2
import seaborn as sns

sns.set(style="white", context="paper")
from cycler import cycler
import os, sys
import glob
from datetime import datetime, timedelta
from itertools import combinations
import base64
from PIL import Image
from io import BytesIO as _BytesIO
import requests
import json
import pickle
from datetime import datetime
from IPython.display import display, Markdown, Latex
from sklearn.metrics import *
import collections
from copy import deepcopy
pd.options.display.max_columns = None
def printm(s): return display(Markdown(s))


In [2]:
casas_dict = pickle.load(open("../cache/casas_prediction_requests.pb",'rb'))
casas_dict.keys()

dict_keys(['csh101', 'csh102', 'csh103', 'csh104', 'csh105', 'csh106', 'csh107', 'csh108', 'csh109', 'csh110', 'csh111', 'csh112', 'csh113', 'csh114', 'csh115', 'csh116', 'csh117', 'csh118', 'csh119', 'csh120', 'csh121', 'csh122', 'csh123', 'csh124', 'csh125', 'csh126', 'csh127', 'csh128', 'csh129', 'csh130'])

In [3]:
casas_dict['csh101'].info()
casas_dict['csh101'].head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 321457 entries, 0 to 321456
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   timestamp   321457 non-null  float64
 1   activities  321457 non-null  object 
dtypes: float64(1), object(1)
memory usage: 7.4+ MB


activity_name,timestamp,activities
0,1.342781e+09,Step_Out
1,1.342781e+09,Step_Out
2,1.342781e+09,Step_Out
3,1.342781e+09,Step_Out
4,1.342781e+09,Step_Out


In [5]:
key='csh101'


In [13]:
df_key = casas_dict[key].copy(deep=True)

In [18]:
df_key.timestamp = df_key.timestamp.astype(int)
df_key.timestamp.max()

1347925995

In [19]:
df_key = df_key.groupby('timestamp',as_index=False).agg({'activities':[lambda x: ','.join(np.unique(list(x)).tolist())]})
df_key.head()


,timestamp,activities
,,<lambda>
0,1342780734,Step_Out
1,1342780737,Step_Out
2,1342780738,Step_Out
3,1342780739,Step_Out
4,1342780742,Step_Out


In [20]:
df_key.columns = ['timestamp','activities']
df_key.activities.unique()

array(['Step_Out', 'Other_Activity', 'Other_Activity,Toilet', 'Toilet',
       'Phone', 'Other_Activity,Personal_Hygiene', 'Personal_Hygiene',
       'Leave_Home', 'Enter_Home', 'Relax',
       'Other_Activity,Sleep_Out_Of_Bed', 'Sleep_Out_Of_Bed',
       'Leave_Home,Other_Activity', 'Enter_Home,Other_Activity',
       'Other_Activity,Relax', 'Drink,Other_Activity', 'Drink',
       'Other_Activity,Watch_TV', 'Watch_TV', 'Dress',
       'Evening_Meds,Other_Activity', 'Evening_Meds', 'Wake_Up', 'Read',
       'Morning_Meds,Other_Activity', 'Morning_Meds',
       'Other_Activity,Read', 'Dress,Other_Activity',
       'Cook_Breakfast,Other_Activity', 'Cook_Breakfast', 'Eat_Breakfast',
       'Bathe', 'Other_Activity,Phone', 'Other_Activity,Step_Out',
       'Cook_Lunch,Other_Activity', 'Cook_Lunch',
       'Eat_Lunch,Other_Activity', 'Eat_Lunch', 'Wash_Lunch_Dishes',
       'Go_To_Sleep,Other_Activity', 'Go_To_Sleep', 'Sleep',
       'Other_Activity,Sleep', 'Bed_Toilet_Transition,Other_Acti

In [21]:
for key in casas_dict.keys():
    casas_dict[key].timestamp = casas_dict[key].timestamp.astype(int)
    casas_dict[key] = casas_dict[key].groupby('timestamp',as_index=False).agg({'activities':[lambda x: ','.join(np.unique(list(x)).tolist())]})
    casas_dict[key].columns = ['timestamp','activities']
    print("Done",":",key)

Done : csh101
Done : csh102
Done : csh103
Done : csh104
Done : csh105
Done : csh106
Done : csh107
Done : csh108
Done : csh109
Done : csh110
Done : csh111
Done : csh112
Done : csh113
Done : csh114
Done : csh115
Done : csh116
Done : csh117
Done : csh118
Done : csh119
Done : csh120
Done : csh121
Done : csh122
Done : csh123
Done : csh124
Done : csh125
Done : csh126
Done : csh127
Done : csh128
Done : csh129
Done : csh130


In [22]:
df_all_casas = None
for key in casas_dict.keys():
    casas_dict[key]['id']= key
    if df_all_casas is None:
        df_all_casas = casas_dict[key].copy(deep=True)
    else:
        df_all_casas = pd.concat([df_all_casas, casas_dict[key]],ignore_index=False)
    print("Done: ", key)

Done:  csh101
Done:  csh102
Done:  csh103
Done:  csh104
Done:  csh105
Done:  csh106
Done:  csh107
Done:  csh108
Done:  csh109
Done:  csh110
Done:  csh111
Done:  csh112
Done:  csh113
Done:  csh114
Done:  csh115
Done:  csh116
Done:  csh117
Done:  csh118
Done:  csh119
Done:  csh120
Done:  csh121
Done:  csh122
Done:  csh123
Done:  csh124
Done:  csh125
Done:  csh126
Done:  csh127
Done:  csh128
Done:  csh129
Done:  csh130


In [23]:
df_all_casas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10256846 entries, 0 to 111858
Data columns (total 3 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   timestamp   int64 
 1   activities  object
 2   id          object
dtypes: int64(1), object(2)
memory usage: 313.0+ MB


In [40]:
df_all_casas.groupby('id').agg({'timestamp':[lambda x:pd.to_datetime(x,unit='s').dt.date.nunique()]}).sum()


timestamp  <lambda>    2394
dtype: int64

In [24]:
df_all_casas.to_csv("../cache/casas_complete_trace.csv",index=False)

In [25]:
extra_dict = pickle.load(open("../cache/extrasensory_prediction_requests.pb",'rb'))
extra_dict.keys()

dict_keys(['00EABED2-271D-49D8-B599-1D4A09240601', '098A72A5-E3E5-4F54-A152-BBDA0DF7B694', '0A986513-7828-4D53-AA1F-E02D6DF9561B', '0BFC35E2-4817-4865-BFA7-764742302A2D', '0E6184E1-90C0-48EE-B25A-F1ECB7B9714E', '1155FF54-63D3-4AB2-9863-8385D0BD0A13', '11B5EC4D-4133-4289-B475-4E737182A406', '136562B6-95B2-483D-88DC-065F28409FD2', '1538C99F-BA1E-4EFB-A949-6C7C47701B20', '1DBB0F6F-1F81-4A50-9DF4-CD62ACFA4842', '24E40C4C-A349-4F9F-93AB-01D00FB994AF', '27E04243-B138-4F40-A164-F40B60165CF3', '2C32C23E-E30C-498A-8DD2-0EFB9150A02E', '33A85C34-CFE4-4732-9E73-0A7AC861B27A', '3600D531-0C55-44A7-AE95-A7A38519464E', '40E170A7-607B-4578-AF04-F021C3B0384A', '481F4DD2-7689-43B9-A2AA-C8772227162B', '4E98F91F-4654-42EF-B908-A3389443F2E7', '4FC32141-E888-4BFF-8804-12559A491D8C', '5119D0F8-FCA8-4184-A4EB-19421A40DE0D', '5152A2DF-FAF3-4BA8-9CA9-E66B32671A53', '59818CD2-24D7-4D32-B133-24C2FE3801E5', '59EEFAE0-DEB0-4FFF-9250-54D2A03D0CF2', '5EF64122-B513-46AE-BCF1-E62AAC285D2C', '61359772-D8D8-480D-B623-7C63

In [28]:
extra_dict['1155FF54-63D3-4AB2-9863-8385D0BD0A13'].head()

activity_name,timestamp,activities
21129,1.464130e+09,"Sitting,Talking,With co-workers"
21130,1.464130e+09,"Sitting,Talking,With co-workers"
21131,1.464130e+09,"Sitting,Talking,With co-workers"
21132,1.464130e+09,"Sitting,Talking,With co-workers"
21133,1.464130e+09,"Sitting,Talking,With co-workers"


In [29]:
df_all_extra = None
for key in extra_dict.keys():
    extra_dict[key]['timestamp']= extra_dict[key]['timestamp'].astype(int)
    extra_dict[key]['id']= key
    if df_all_extra is None:
        df_all_extra = extra_dict[key].copy(deep=True)
    else:
        df_all_extra = pd.concat([df_all_extra, extra_dict[key]],ignore_index=False)
    print("Done: ", key)

Done:  00EABED2-271D-49D8-B599-1D4A09240601
Done:  098A72A5-E3E5-4F54-A152-BBDA0DF7B694
Done:  0A986513-7828-4D53-AA1F-E02D6DF9561B
Done:  0BFC35E2-4817-4865-BFA7-764742302A2D
Done:  0E6184E1-90C0-48EE-B25A-F1ECB7B9714E
Done:  1155FF54-63D3-4AB2-9863-8385D0BD0A13
Done:  11B5EC4D-4133-4289-B475-4E737182A406
Done:  136562B6-95B2-483D-88DC-065F28409FD2
Done:  1538C99F-BA1E-4EFB-A949-6C7C47701B20
Done:  1DBB0F6F-1F81-4A50-9DF4-CD62ACFA4842
Done:  24E40C4C-A349-4F9F-93AB-01D00FB994AF
Done:  27E04243-B138-4F40-A164-F40B60165CF3
Done:  2C32C23E-E30C-498A-8DD2-0EFB9150A02E
Done:  33A85C34-CFE4-4732-9E73-0A7AC861B27A
Done:  3600D531-0C55-44A7-AE95-A7A38519464E
Done:  40E170A7-607B-4578-AF04-F021C3B0384A
Done:  481F4DD2-7689-43B9-A2AA-C8772227162B
Done:  4E98F91F-4654-42EF-B908-A3389443F2E7
Done:  4FC32141-E888-4BFF-8804-12559A491D8C
Done:  5119D0F8-FCA8-4184-A4EB-19421A40DE0D
Done:  5152A2DF-FAF3-4BA8-9CA9-E66B32671A53
Done:  59818CD2-24D7-4D32-B133-24C2FE3801E5
Done:  59EEFAE0-DEB0-4FFF-9250-5

In [30]:
df_all_extra.to_csv("../cache/extrasensory_complete_trace.csv",index=False)

In [41]:
df_all_extra.groupby('id').agg({'timestamp':[lambda x:pd.to_datetime(x,unit='s').dt.date.nunique()]}).sum()


activity_name          
timestamp      <lambda>    471
dtype: int64

In [32]:
df_all_extra.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307534 entries, 0 to 307533
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   timestamp   307534 non-null  int64 
 1   activities  307534 non-null  object
 2   id          307534 non-null  object
dtypes: int64(1), object(2)
memory usage: 9.4+ MB


In [33]:
df_all_extra.head(100)

activity_name,timestamp,activities,id
0,1444079161,"Sitting,In a meeting,With co-workers",00EABED2-271D-49D8-B599-1D4A09240601
1,1444079221,"Sitting,In a meeting,With co-workers",00EABED2-271D-49D8-B599-1D4A09240601
2,1444079281,"Sitting,In a meeting,With co-workers",00EABED2-271D-49D8-B599-1D4A09240601
3,1444079341,"Sitting,In a meeting,With co-workers",00EABED2-271D-49D8-B599-1D4A09240601
4,1444079431,"Sitting,In a meeting,With co-workers",00EABED2-271D-49D8-B599-1D4A09240601
...,...,...,...
95,1444084892,"Sitting,Computer work",00EABED2-271D-49D8-B599-1D4A09240601
96,1444084952,"Sitting,Computer work",00EABED2-271D-49D8-B599-1D4A09240601
97,1444085012,"Sitting,Computer work",00EABED2-271D-49D8-B599-1D4A09240601
98,1444085072,"Sitting,Computer work",00EABED2-271D-49D8-B599-1D4A09240601


In [42]:
df_all_casas.head()

,timestamp,activities,id
0,1342780734,Step_Out,csh101
1,1342780737,Step_Out,csh101
2,1342780738,Step_Out,csh101
3,1342780739,Step_Out,csh101
4,1342780742,Step_Out,csh101


In [44]:
df_sm_casas = df_all_casas[df_all_casas.id.isin(['csh101','csh102','csh103','csh104','csh105'])]
df_sm_casas.head()

,timestamp,activities,id
0,1342780734,Step_Out,csh101
1,1342780737,Step_Out,csh101
2,1342780738,Step_Out,csh101
3,1342780739,Step_Out,csh101
4,1342780742,Step_Out,csh101


In [2]:
df_onto_casas = pd.read_csv("../cache/trace_results/casas_onto_trace.csv")
df_onto_casas = df_onto_casas[df_onto_casas.id.isin(['csh101','csh102','csh103','csh104','csh105'])]
df_onto_casas = df_onto_casas[['id','timestamp','tao_prediction']]
df_onto_casas.head()

,id,timestamp,tao_prediction
0,csh101,1342780734,GoingOut
1,csh101,1342780737,GoingOut
2,csh101,1342780738,GoingOut
3,csh101,1342780739,GoingOut
4,csh101,1342780742,GoingOut


In [ ]:
casas_tp_dict = pickle.load(open("../cache/trace_results/casas_temporal_trace.pb","rb"))


In [ ]:
df_temp_casas = None
for key in ['csh101','csh102','csh103','csh104','csh105']:
    if df_temp_casas is None:
        df_temp_casas = casas_tp_dict[key].copy(deep=True)
    else:
        df_temp_casas = pd.concat([df_temp_casas, casas_tp_dict[key]],ignore_index=True)
df_temp_casas.head()

In [ ]:
df_ctx_casas = pd.merge(df_onto_casas, df_temp_casas, on =['id','timestamp'])